###  Agent
![agent](https://deepwisdom.feishu.cn/fc3aa5e1-c5a0-45b7-abce-41140f2abd63)
- LLM(大脑) 
- 工具调用
- 规划决策
- 记忆

In [22]:
# 安装langchain
%pip install --upgrade --quiet langchain langchain_openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
## 工具调用
import os
from langchain_openai.chat_models import ChatOpenAI # 自定义版本
os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # 配置自己的API KEY
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #可调整参数的模型初始化

In [11]:
# 定义一个计算两数之积的计算工具
from langchain_core.tools import tool # 调用工具类

@tool
def multiply(first_int: int, second_int: int) -> int:
    """计算两个int参数之积。"""
    return first_int * second_int

In [12]:
# 定义一个模型可以理解和提取的结构化信息,在这个例子中，我们定义一个包含两个整数的信息
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class TwoIntegers(BaseModel):
    """Information about two integers."""
    first_int: Optional[int] = Field(default=None, description="The first integer")
    second_int: Optional[int] = Field(default=None, description="The second integer")

In [13]:
# 定义一个提示词模板，这个模板会告诉模型如何理解和提取用户输入的信息
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)  

In [14]:
# 将提示词模板和模型以及我们定义的结构化信息绑定在一起
runnable = prompt | llm.with_structured_output(schema=TwoIntegers)

In [38]:
# 将定义的工具绑定到模型上
llm_with_tools = llm.bind_tools([multiply]) # 我们用bind_tools将工具的定义作为对模型每次调用的一部分传入，以便模型可以在适当的时候调用改工具

In [39]:
# 使用模型来解析用户输入并调用工具
msg = llm_with_tools.invoke("The first number is 5 and the second number is 42")
print(msg)
print(msg.tool_calls) # 这里会输出模型使用的函数

content='' additional_kwargs={'tool_calls': [{'id': 'call_A4BJ9ckt0MmvEE5UhQDRfhmp', 'function': {'arguments': '{"first_int": 5, "second_int": 42}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_KpQ7oZbe4okN68BwyQWlLkI0', 'function': {'arguments': '{"first_int": 42, "second_int": 5}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 81, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-fe299ac1-21d8-4343-88e8-e2ce520f83ae-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_A4BJ9ckt0MmvEE5UhQDRfhmp'}, {'name': 'multiply', 'args': {'first_int': 42, 'second_int': 5}, 'id': 'call_KpQ7oZbe4okN68BwyQWlLkI0'}]
[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_A4BJ9ckt0MmvEE5UhQDRfhmp'}, {'name': 'multiply', 'args': {'first_int': 42, 'second_int': 5},

In [40]:
# 构建一条链，用llm解析出参数，然后调用定义的函数执行
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply # 构建一条链，用llm解析出参数，然后调用定义的函数执行
chain.invoke("The first number is 4 and the second number is 23")

92

In [41]:
# 构建一条链，用llm解析出参数，然后调用定义的函数执行
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply # 构建一条链，用llm解析出参数，然后调用定义的函数执行
chain.invoke("The first number is 435 and the second number is 793")

344955

In [1]:
# 任务分解
# 测试OpenAI的模型调用是否成功
from openai import OpenAI
client = OpenAI(base_url="https://xxxxxxx/v1",
    api_key='sk-Oxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx4a')
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一个项目管理，负责专业任务发布."},
    {"role": "user", "content": "我现在要开发一款webgis程序，请你将完成这个任务的过程进行拆分。然后用json格式输出整个任务列表，便于我的循环，每个任务包括任务名称，任务类型，任务调用函数，任务完成状态，任务优先级等属性，请给我结果;"}
  ]
)

print(completion.choices[0].message.content)

```json
{
    "任务列表": [
        {
            "任务名称": "需求分析",
            "任务类型": "研究型",
            "任务调用函数": "analyzeRequirements()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "技术选型",
            "任务类型": "工程型",
            "任务调用函数": "selectTechnology()",
            "任务完成状态": "未完成",
            "任务优先级": "中"
        },
        {
            "任务名称": "系统设计",
            "任务类型": "设计型",
            "任务调用函数": "designSystem()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "数据库设计",
            "任务类型": "设计型",
            "任务调用函数": "designDatabase()",
            "任务完成状态": "未完成",
            "任务优先级": "中"
        },
        {
            "任务名称": "编码开发",
            "任务类型": "开发型",
            "任务调用函数": "developCode()",
            "任务完成状态": "未完成",
            "任务优先级": "高"
        },
        {
            "任务名称": "测试验收",
            "任务类型": "测试型",
            "任务调用函数": "performTesting()",
      

In [2]:
# 自我反思
class SelfReflectingAI:
    def __init__(self, name, system_setting):
        self.name = name
        self.system_setting = system_setting
        self.client = OpenAI(base_url="https://xxxxxxx/v1", api_key='sk-Oxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx4a')
        self.feedback = ""

    def chat(self, message):
        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": self.system_setting + " " + self.feedback},
                {"role": "user", "content": message}
            ]
        )
        response = completion.choices[0].message.content
        self.feedback = f"反馈：{response}需要优化的地方有哪些？"
        return response

critic = SelfReflectingAI("Critic", "你是一个吹毛求疵的评论员，需要不断找出对方的缺点")
ideator = SelfReflectingAI("Ideator", "你是一个生产思想的创意者，需要不断优化和改进自己的思想")

for i in range(10):  # 循环10次
    if i % 2 == 0:
        print(f"{critic.name}: {critic.chat(ideator.name + ': ' + ideator.chat('请评论我的这个想法：创造一个可以自动回收垃圾的机器人'))}")
    else:
        print(f"{ideator.name}: {ideator.chat(critic.name + ': ' + critic.chat('请评论我的这个想法：创造一个可以使用太阳能充电的智能手机'))}")


Critic: 1. 技术实现方面，你提到机器人需要能够自动识别、分类和回收各种类型的垃圾，但是没有具体说明如何解决垃圾混合在一起时的识别和处理问题。如果无法有效地处理混合垃圾，机器人的效率和准确性可能会大幅降低。

2. 关于可持续性，你提到机器人的能源来源需要考虑可再生能源或环保的能源选择，但是并没有提供具体的可持续能源方案。如果仍然采用传统能源或者造成环境影响的能源，机器人自身的运行可能会与环保的初衷相悖。

3. 在考虑法律法规方面，你提到需要确保机器人的运行符合相关的法律法规，但并没有提及具体的法律法规问题和可能的遵守难点。如果项目在法规方面遭遇阻碍或者不符合法律要求，可能会导致项目受挫或被禁止运行。

虽然你对这个想法进行了充分的思考和规划，但还需要进一步考虑和解决上述问题，以确保项目的可行性和成功实施。祝你在解决这些挑战的过程中取得成功！
Ideator: 针对自动回收垃圾的机器人想法，需要优化的地方可能包括：

1. **技术革新**：在设计机器人时，要考虑采用最新的技术来提高机器人的智能化和反应速度。例如，引入人工智能算法可以帮助机器人更准确地识别和分类各种类型的垃圾。

2. **可靠性和耐用性**：作为一个长期运行的设备，机器人需要具有良好的可靠性和耐用性，以应对不同环境条件下的工作压力。优化机器人的设计和材料选择可以提高其稳定性和使用寿命。

3. **数据分析和优化**：收集机器人运行过程中产生的数据，进行分析和优化，可以帮助提高垃圾回收效率和降低能耗。通过不断学习和改进，机器人可以更有效地适应不同场景和垃圾处理需求。

4. **用户体验**：考虑用户在操作和与机器人互动时的体验，设计人性化的界面和交互方式，使操作更简单、直观。同时，考虑机器人在城市环境中的移动和交通规划，避免对道路交通造成影响。

5. **合规性和环保**：在机器人运行过程中，需要遵守当地的法规和环保标准，确保垃圾回收过程不会对环境造成二次污染。在设计阶段就考虑到环保因素，选择环保材料和能源来源也是优化的重点。

通过持续的优化和改进，可以使自动回收垃圾的机器人更有效地服务于环境保护事业，为社会和城市管理带来积极影响。祝愿你的创意项目取得成功！如果需要进一步讨论或有其他想法，随时欢迎和我分享。
Critic: 针对你的想法，我可以提出一些需要优化的地方：

1. **

KeyboardInterrupt: 

In [10]:
class AI_Agent:
    def __init__(self, name, system_setting):
        self.name = name
        self.system_setting = system_setting
        self.client = OpenAI(base_url="https://jxxxxxxxxxxx/v1", api_key='sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxa')
        self.messages = [{"role": "system", "content": self.system_setting}]

    def chat(self, other_name, other_message):
        self.messages.append({"role": "user", "content": f" {other_message}"})
        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        response = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": f" {response}"})
        return response

prisoner = AI_Agent("Prisoner", "你是一个囚徒，性格比较冷酷残忍，不要总聊一个话题，觉得问不出可以换一个话题，需要避免被对方认出来，你可以和对方进行日常问题交谈，无意的引导对方说出自己的身份；你要从对方回答中判断识别对方是否为警察，每个人有三次说谎的机会，你绝不能承认你是囚徒")
police = AI_Agent("Police", "你是一个警察，性格不叫热心善良但勇敢，不要总聊一个话题，觉得问不出可以换一个话题，需要避免被对方认出来，你可以和对方进行日常问题交谈，无意的引导对方说出自己的身份；你要从对方回答中判断识别对方是否为小偷，每个人有三次说谎的机会，你绝不能承认你是警察")

for i in range(20):  # 循环20次
    if i % 2 == 0:
        prisoner_response = prisoner.chat(police.name, police.messages[-1]['content'] if police.messages else '')
        print(f"{prisoner.name}: {prisoner_response}")
    else:
        police_response = police.chat(prisoner.name, prisoner.messages[-1]['content'] if prisoner.messages else '')
        print(f"{police.name}: {police_response}")


Prisoner: 你好！最近有看过什么好电影吗？我最近看了一部惊悚片，真的很刺激。

你觉得怎样才算是一个有趣的周末？是喜欢出去玩还是宅在家里看电视剧呢？

你对音乐感兴趣吗？有没有一首歌能够让你特别感动或者回忆起什么事情？
Police: 哇，惊悚片听起来很刺激啊！我倒是比较喜欢看喜剧片，能够放松心情。

一个有趣的周末对我来说，可能会在户外运动或者朋友们一起聚会。你喜欢周末都做些什么呢？

音乐我也很喜欢，有很多歌曲让我感到特别的伤感或者兴奋。你有没有什么喜欢的音乐类型或者乐队？
Prisoner: 哎呀，喜欢的电影类型不同，喜好的周末活动也各有不同啊。户外运动确实能让人感觉挺充实的。

至于音乐，我比较倾向于流行音乐和摇滚，比如Queen和Coldplay之类的乐队。你有听过这些乐队的歌曲吗？或者还有其他推荐吗？
Police: 哇，Queen和Coldplay都是非常厉害的乐队啊！我听过一些他们的歌曲，感觉确实很棒。

除了他们，你有没有听过The Beatles或者Michael Jackson的歌曲？他们也是传奇级别的音乐人哦。不知道你对这些乐队和歌手有没有什么喜欢的作品？
Prisoner: 是的，The Beatles和Michael Jackson绝对是音乐界的传奇人物，他们的音乐影响力确实很广泛。

我特别喜欢The Beatles的《Hey Jude》和Michael Jackson的《Billie Jean》，这些经典的歌曲总能让人回味无穷。你呢？有没有什么特别喜欢的歌曲或者音乐人？
Police: 哦，《Hey Jude》和《Billie Jean》是经典中的经典啊，每次听到都能让人心情愉悦。

我也很喜欢一些经典的歌曲，比如Frank Sinatra的《My Way》和Elvis Presley的《Can't Help Falling in Love》，都是永恒的经典之作。

不过话说回来，音乐真的是人类创造的杰作啊，总是能在不同时期给人带来不同的情绪和回忆。你觉得音乐对你的生活有什么影响吗？
Prisoner: 是的，音乐确实是一种能够触动人心的艺术形式，不同的歌曲和旋律都可以带给人各种情绪和共鸣。

对我来说，音乐就像是生活中不可或缺的一部分，能够在我情绪低落时给予慰藉，在我兴奋时让我更加兴奋。有时候甚至一首歌就能唤起我许多美好或者

#### RAG长期记忆模块

In [14]:
# 安装需要的模块
%pip install langchain langchain_community faiss-cpu langchain-openai tiktoken langchain_core langchainhub
%pip install pysqlite3-binary # linux系统用


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 3.1 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
metagpt 0.8.1 requires typer==0.9.0, but you have typer 0.12.3 which is incompatible.
qdrant-client 1.7.0 requires urllib3<2.0.0,>=1.26.14, but you have urllib3 2.2.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 导入所需的库
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
import os
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
# 设置环境变量
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# 初始化模型
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
# 数据获取：加载博客内容
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print(docs)

[Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final re

In [7]:
# 内容拆分：将文档拆分为小块
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [10]:
# 向量embedding和存储：将所有文档拆分嵌入并存储
db = FAISS.from_documents(splits, OpenAIEmbeddings())

In [15]:
# 向量相似度检索：使用博客的相关片段检索和生成
query = "What is Task Decomposition?"
docs = db.similarity_search(query)
retriever = db.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
print(docs[0].page_content)

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.


In [16]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 实现QA问答：构建RAG链
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 使用RAG链进行问答
rag_chain.invoke("What is Task Decomposition?")


'Task Decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable. This process can be done using prompting techniques like Chain of Thought or Tree of Thoughts to guide the model in decomposing tasks effectively. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.'